In [1]:
import parsl
from parsl.app.app import python_app, bash_app
import time
import os

print("Parsl version: {}".format(parsl.__version__))
print("Parsl file source: {}".format(parsl.__file__))

Parsl version: 0.7.2-a0
Parsl file source: /home/ubuntu/anaconda3/envs/remote_ix_py3.6/lib/python3.6/site-packages/parsl/__init__.py


Enable only for logging. Please remove prior to demo time

In [2]:
# parsl.set_stream_logger()

#### Configurations

Here we configure Parsl to use the Theta Supercomputer at Argonne National Lab, and Cori at NERSC.

##### Authentication
Both systems use two factor authentication, which Parsl supports via an interactive SSH channel that
requires the user to enter the 2FA token string as per the facilities setup when prompted.

##### Execution


##### Data Transport


In [3]:
from parsl.config import Config

from parsl.providers import SlurmProvider, CobaltProvider
from parsl.executors import HighThroughputExecutor
from parsl.channels import SSHInteractiveLoginChannel

config = Config(
    executors=[
        HighThroughputExecutor(
            label='theta',
            max_workers=4,
            address='try.parsl-project.org',
            interchange_address='thetalogin6',            
            provider=CobaltProvider(                
                channel=SSHInteractiveLoginChannel(
                    hostname="thetalogin6.alcf.anl.gov",
                    username="yadunand",                         # MUST SET PER USER
                    script_dir="/home/yadunand/parsl_scripts"                    
                ),
                queue="debug-flat-quad",
                init_blocks=1,                
                worker_init='source /home/yadunand/setup_parsl_0.7.2.sh',
                account='CSC249ADCD01',                          # MUST SET PER USER
                cmd_timeout=120
            ),            
        ),        
        HighThroughputExecutor(
            label="cori",
            worker_debug=False,
            address='try.parsl-project.org',
            interchange_address='cori03-224.nersc.gov',            
            provider=SlurmProvider(
                partition='debug',  # Replace with partition name
                # channel=SSHChannel(
                channel=SSHInteractiveLoginChannel(
                    hostname='cori03-224.nersc.gov',
                    username='yadunand',                         # MUST SET PER USER
                    script_dir='/global/homes/y/yadunand/parsl_scripts',
                ),                
                init_blocks=1,                
                scheduler_options="#SBATCH --constraint=knl,quad,cache",
                # scheduler_options="#SBATCH --constraint=haswell",
                worker_init='source ~/setup_parsl_0.7.2.sh',                
            ),
        )
    ],
)

/home/ubuntu/anaconda3/envs/remote_ix_py3.6/lib/python3.6/site-packages/paramiko/ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo
/home/ubuntu/anaconda3/envs/remote_ix_py3.6/lib/python3.6/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
/home/ubuntu/anaconda3/envs/remote_ix_py3.6/lib/python3.6/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePubl

Enter thetalogin6.alcf.anl.gov Logon password :········
Enter cori03-224.nersc.gov Logon password :········


In [4]:
parsl.load(config)

In [5]:
@python_app(executors=['theta'])
def platform_theta(sleep=10, stdout=None):
    import platform
    import time
    time.sleep(sleep)
    return platform.uname()

@python_app(executors=['cori'])
def platform_cori(sleep=10, stdout=None):
    import platform
    import time
    time.sleep(sleep)
    return platform.uname()

@python_app
def double(x):
    return x * 2

In [6]:
theta_p = platform_theta()
cori_p = platform_cori()

In [7]:
# print("Got result: {}".format(x.result())
print("From theta: ", theta_p.result())
print("From cori: ", cori_p.result())

From theta:  uname_result(system='Linux', node='nid00001', release='4.4.103-6.38_4.0.104-cray_ari_c', version='#1 SMP Thu Jan 17 19:44:04 UTC 2019 (5395680)', machine='x86_64', processor='x86_64')
From cori:  uname_result(system='Linux', node='nid02517', release='4.4.103-6.38_4.0.153-cray_ari_c', version='#1 SMP Thu Nov 1 16:05:05 UTC 2018 (6ef8fef)', machine='x86_64', processor='x86_64')


In [ ]:
time.sleep(300)

In [9]:
parsl.dfk().cleanup()